In [10]:
import numpy as np
import pandas as pd
#import seaborn as sns
import scipy.stats as stats
import matplotlib.pyplot as plt
# from statsmodels.stats.proportion import proportions_ztest
# from statsmodels.stats.proportion import proportions_chisquare
from IPython.display import display, HTML
from scipy.stats import kruskal

import sys
import logging
logging.basicConfig(level=logging.CRITICAL)

from pytqs import tqs
import bytedtqs

In [11]:
# appId = "eL38NWjpAMXlC0ifCoo71wk27zE6CS9YQismEcwVH5oNKbJf"
# appKey = "VeeOKtXLxjGWE2OyIGkuqWRT33isyxtHH1xlxakeFzId6SMB"
# user_id = 'wuweiwei.1012'

In [12]:
def two_proprotions_test(success_a, size_a, success_b, size_b):
    prop_a = success_a / size_a
    prop_b = success_b / size_b
    prop_pooled = (success_a + success_b) / (size_a + size_b)
    var = prop_pooled * (1 - prop_pooled) * (1 / size_a + 1 / size_b)
    zscore = np.abs(prop_b - prop_a) / np.sqrt(var)
    one_side = 1 - stats.norm(loc = 0, scale = 1).cdf(zscore)
    pvalue = one_side * 2
    return zscore, pvalue

### 获取实验数据

In [13]:
all_data = pd.read_csv('/home/tiger/archived-data/aeolus-data/20210310/13/39648599-课程实验-复习课题目顺序-查询60.csv', encoding = 'gb18030')

In [14]:
all_data.head()

,abtest_dt,user_group,user_id,milestone_name,finish_review_yn,finish_review_2nd_yn,ttl_ruler_exe,correct_exe,ttl_mastery_degree_d1,ttl_kp_num_d1,...,ttl_mastery_degree_d10,ttl_kp_num_d10,ttl_mastery_degree_d11,ttl_kp_num_d11,ttl_mastery_degree_d12,ttl_kp_num_d12,ttl_mastery_degree_d13,ttl_kp_num_d13,ttl_mastery_degree_d14,ttl_kp_num_d14
0,2021-02-27,实验组,3223108,B1中,0,0,0,0,0.0,0,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0
1,2021-02-09,实验组,2455666821,A2初,0,0,0,0,0.0,0,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0
2,2021-02-01,实验组,3101112775,未定级,0,0,0,0,0.0,0,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0
3,2021-02-11,实验组,3102346712,未定级,0,0,0,0,0.0,0,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0
4,2021-01-22,对照组,3141714654,C2,0,0,0,0,0.0,0,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0


In [15]:
len(all_data)

1242425

In [16]:
all_data.columns

Index(['abtest_dt', 'user_group', 'user_id', 'milestone_name',
       'finish_review_yn', 'finish_review_2nd_yn', 'ttl_ruler_exe',
       'correct_exe', 'ttl_mastery_degree_d1', 'ttl_kp_num_d1',
       'ttl_mastery_degree_d2', 'ttl_kp_num_d2', 'ttl_mastery_degree_d3',
       'ttl_kp_num_d3', 'ttl_mastery_degree_d4', 'ttl_kp_num_d4',
       'ttl_mastery_degree_d5', 'ttl_kp_num_d5', 'ttl_mastery_degree_d6',
       'ttl_kp_num_d6', 'ttl_mastery_degree_d7', 'ttl_kp_num_d7',
       'ttl_mastery_degree_d8', 'ttl_kp_num_d8', 'ttl_mastery_degree_d9',
       'ttl_kp_num_d9', 'ttl_mastery_degree_d10', 'ttl_kp_num_d10',
       'ttl_mastery_degree_d11', 'ttl_kp_num_d11', 'ttl_mastery_degree_d12',
       'ttl_kp_num_d12', 'ttl_mastery_degree_d13', 'ttl_kp_num_d13',
       'ttl_mastery_degree_d14', 'ttl_kp_num_d14'],
      dtype='object')

In [24]:
### 预处理数据
#1. 填充空值数据
#2. 增加新字段

# all_data = all_data[~all_data.milestone_name.isin(['未定级'])]

# review_test_data.head()


#### 全量用户 

In [24]:
def ab_summary_data(data, user_group):
    test_summary = pd.DataFrame(columns = ["User_Group", "Metrics", "Control", "Test", "Delta", "p_value","Significance"])
    
    test_data = data.loc[data.user_group == '实验组']
    ctrl_data = data.loc[data.user_group == '对照组']

    dnu_t = len(test_data)
    dnu_c = len(ctrl_data)

    # 新增用户
    test_summary = test_summary.append({"Metrics": '实验用户', "Control": '{:,}'.format(dnu_c), "Test": '{:,}'.format(dnu_t), "Delta": '-', "p_value": '-', "Significance": '-'}, ignore_index = True)

    # 复习课首课完课
    fst_fin_t = sum(test_data.finish_review_yn)
    fst_fin_c = sum(ctrl_data.finish_review_yn)
    
    fst_fin_rate_t = round(fst_fin_t / dnu_t, 4)
    fst_fin_rate_c = round(fst_fin_c / dnu_c, 4)
    fst_fin_rate_delta = fst_fin_rate_t / fst_fin_rate_c - 1 if fst_fin_rate_c >0 else np.nan
    
    fst_fin_rate_p_value = two_proprotions_test(fst_fin_t, dnu_t, fst_fin_c, dnu_c)[1]
    
    fst_fin_rate_sig =  "Significant" if fst_fin_rate_p_value < 0.1 else "Non-significant"
    test_summary = test_summary.append({"Metrics": '首次复习课推送->完课', "Control": "{:.2%}".format(fst_fin_rate_c), "Test": "{:.2%}".format(fst_fin_rate_t), "Delta": "{:.2%}".format(fst_fin_rate_delta), "p_value": round(fst_fin_rate_p_value, 4), "Significance": fst_fin_rate_sig}, ignore_index = True)

    # 复习课次课完课
    fst_fin_t = sum(test_data.finish_review_2nd_yn)
    fst_fin_c = sum(ctrl_data.finish_review_2nd_yn)
    
    fst_fin_rate_t = round(fst_fin_t / dnu_t, 4)
    fst_fin_rate_c = round(fst_fin_c / dnu_c, 4)
    fst_fin_rate_delta = fst_fin_rate_t / fst_fin_rate_c - 1 if fst_fin_rate_c >0 else np.nan
    
    fst_fin_rate_p_value = two_proprotions_test(fst_fin_t, dnu_t, fst_fin_c, dnu_c)[1]
    
    fst_fin_rate_sig =  "Significant" if fst_fin_rate_p_value < 0.1 else "Non-significant"
    test_summary = test_summary.append({"Metrics": '次课复习课推送->完课', "Control": "{:.2%}".format(fst_fin_rate_c), "Test": "{:.2%}".format(fst_fin_rate_t), "Delta": "{:.2%}".format(fst_fin_rate_delta), "p_value": round(fst_fin_rate_p_value, 4), "Significance": fst_fin_rate_sig}, ignore_index = True)

    
    # 新增用户
    ttl_ruler_t = sum(test_data.ttl_ruler_exe)
    ttl_ruler_c = sum(ctrl_data.ttl_ruler_exe)
    
    test_summary = test_summary.append({"Metrics": '标尺题总数', "Control": '{:,}'.format(ttl_ruler_c), "Test": '{:,}'.format(ttl_ruler_t), "Delta": '-', "p_value": '-', "Significance": '-'}, ignore_index = True)

    # 标尺题答题正确率

    correct_ruler_t = sum(test_data.correct_exe)
    correct_ruler_c = sum(ctrl_data.correct_exe)
    
    ruler_corr_rate_t = round(correct_ruler_t / ttl_ruler_t, 4)
    ruler_corr_rate_c = round(correct_ruler_c / ttl_ruler_c, 4)
    ruler_corr_rate_delta = ruler_corr_rate_t / ruler_corr_rate_c - 1 if ruler_corr_rate_c >0 else np.nan
    
    ruler_corr_rate_p_value = two_proprotions_test(correct_ruler_t, ttl_ruler_t, correct_ruler_c, ttl_ruler_c)[1]
    
    ruler_corr_rate_sig =  "Significant" if ruler_corr_rate_p_value < 0.1 else "Non-significant"
    test_summary = test_summary.append({"Metrics": '标尺题正确率', "Control": "{:.2%}".format(ruler_corr_rate_c), "Test": "{:.2%}".format(ruler_corr_rate_t), "Delta": "{:.2%}".format(ruler_corr_rate_delta), "p_value": round(ruler_corr_rate_p_value, 4), "Significance":ruler_corr_rate_sig}, ignore_index = True)

    # 
#     fst_dur_sum_t = sum(test_data.duration_1st_lesson) / 60
#     fst_dur_sum_c = sum(ctrl_data.duration_1st_lesson) / 60
    
#     fst_duration_t = round(fst_dur_sum_t / dnu_t, 4)
#     fst_duration_c = round(fst_dur_sum_c / dnu_c, 4)
#     fst_duration_delta = fst_duration_t / fst_duration_c - 1 if fst_duration_c >0 else np.nan
    
#     fst_duration_p_value = stats.ttest_ind(test_data.duration_1st_lesson/60, ctrl_data.duration_1st_lesson/60, equal_var = False).pvalue
    
#     fst_duration_sig =  "Significant" if fst_duration_p_value < 0.1 else "Non-significant"
#     test_summary = test_summary.append({"Metrics": '首课学习时长：mins', "Control": str(round(fst_duration_c,2)), "Test": str(round(fst_duration_t,2)), "Delta": "{:.2%}".format(fst_duration_delta), "p_value": round(fst_duration_p_value, 4), "Significance": fst_duration_sig}, ignore_index = True)
   
    test_summary['User_Group'] = user_group
    
    return test_summary

In [25]:
output = ab_summary_data(all_data, '全量用户')
output.style.apply(lambda x: ["background: green" if v == 'Significant' else "" for v in x], axis = 1).hide_index()

User_Group,Metrics,Control,Test,Delta,p_value,Significance
全量用户,实验用户,"621,113","621,312",-,-,-
全量用户,首次复习课推送->完课,0.94%,0.96%,2.13%,0.299600,Non-significant
全量用户,次课复习课推送->完课,1.42%,1.46%,2.82%,0.103500,Non-significant
全量用户,标尺题总数,"47,336","48,412",-,-,-
全量用户,标尺题正确率,74.32%,74.81%,0.66%,0.084200,Significant


##### 分等级

In [27]:
group_list = ['A1初', 'A2初']
output = pd.DataFrame(columns = ["User_Group", "Metrics", "Control", "Test", "Delta", "p_value","Significance"])

for i in range(0,len(group_list)):
    user_group = group_list[i]
    group_data = all_data[all_data.milestone_name == user_group]
    output = output.append(ab_summary_data(group_data, user_group))

output.reset_index(drop=True).style.apply(lambda x: ["background: green" if v == 'Significant' else "" for v in x], axis = 1).hide_index()

User_Group,Metrics,Control,Test,Delta,p_value,Significance
A1初,实验用户,"97,245","97,806",-,-,-
A1初,首次复习课推送->完课,0.37%,0.39%,5.41%,0.322700,Non-significant
A1初,次课复习课推送->完课,0.70%,0.76%,8.57%,0.084700,Significant
A1初,标尺题总数,"9,049","9,248",-,-,-
A1初,标尺题正确率,70.85%,71.12%,0.38%,0.686900,Non-significant
A2初,实验用户,"49,005","49,501",-,-,-
A2初,首次复习课推送->完课,1.34%,1.24%,-7.46%,0.146700,Non-significant
A2初,次课复习课推送->完课,2.33%,2.27%,-2.58%,0.531900,Non-significant
A2初,标尺题总数,"22,715","22,674",-,-,-
A2初,标尺题正确率,75.20%,76.86%,2.21%,0.000000,Significant
